In [29]:
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
import pytz

In [2]:
market = Market()
strategy = Strategy()

In [6]:
market.connect()
prices = market.retrieve("prices")
market.disconnect()

In [7]:
strategy.connect()
weekly_sim = strategy.retrieve("weekly_sim")
strategy.disconnect()

In [8]:
prices["date"] = pd.to_datetime(prices["date"])
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [32]:
seats = 11
strategy.connect()
for number_of_training_weeks in tqdm(range(14,68,14)):
    for weekly_model_training_year in tqdm(range(1,6)):
        for weekly_category_training_year in tqdm(range(1,5)):
            for value in [True,False]:
                current_weekly_sim = weekly_sim[(weekly_sim["mty"]==weekly_model_training_year) & \
                                               (weekly_sim["tw"]==number_of_training_weeks) & \
                                               (weekly_sim["cty"]==weekly_category_training_year)]
                final_sim = prices.merge(current_weekly_sim,on=["year","quarter","week","ticker"],how="left").dropna().reset_index()
                final_sim["weekly_delta"] = (final_sim["weekly_price_regression_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]
                if not value:
                    final_sim["weekly_delta"] = final_sim["weekly_delta"] * -1
                final_sim.rename(columns={"score":"weekly_score"},inplace=True)
                start = final_sim["date"].min()
                end = final_sim["date"].max()
                date = start
                final_sim.sort_values("date",ascending=True,inplace=True)
                for delta in range(0,15,5):
                    d = float(delta/100)
                    for score in range(60,100,10):
                        date = start
                        while date <= end:
                            req = float(score/100)
                            if date.weekday() > 4:
                                date = date + timedelta(days=1)
                            try:
                                todays_sim = final_sim[(final_sim["date"]==date) 
                                                       & (final_sim["weekly_delta"] > d) \
                                                      & (final_sim["weekly_score"] >= req)].sort_values("weekly_delta",ascending=False)
                                if todays_sim.index.size < 1:
                                    date = date + timedelta(days=1)
                                else:
                                    for seat in range(min(seats,todays_sim.index.size)):
                                        try:
                                            trade = todays_sim.iloc[seat]
                                            ticker = trade["ticker"]
                                            buy_price = trade["adjClose"]
                                            exit_price = buy_price * (1+d)
                                            max_days = min(14,(datetime(date.year,12,24).astimezone(pytz.utc)-date).days)
                                            max_date = date + timedelta(days=max_days)
                                            exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                                (final_sim["date"] > date) & \
                                                              (final_sim["date"] <= max_date) & \
                                                              (final_sim["adjClose"] >= exit_price)]
                                            if exits.index.size < 1:
                                                exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                              (final_sim["year"] == date.year) & \
                                                            (final_sim["date"] >max_date)
                                                              (final_sim["adjClose"] >= buy_price)].sort_values("date",ascending=False)
                                                if exits.index.size > 0:
                                                    sell_trade = exits.iloc[0]
                                                else:
                                                    exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                              (final_sim["year"] == date.year)].sort_values("date")
                                                    sell_trade = exits.iloc[exits.index.size -1]
                                            else:
                                                sell_trade = exits.iloc[0]
                                            trade["sell_date"] = sell_trade["date"]
                                            trade["sell_price"] = sell_trade["adjClose"]
                                            trade["delta"] = (trade["sell_price"] - trade["adjClose"]) / trade["adjClose"]
                                            trade["score"] = score
                                            trade["d"] = delta
                                            trade["wcty"] = weekly_category_training_year
                                            trade["wmty"] = weekly_model_training_year
                                            trade["tw"] = number_of_training_weeks
                                            trade["value"] = value
                                            trade["seat"] = seat
                                            strategy.store("trades",pd.DataFrame([trade]))
                                            date = sell_trade["date"] + timedelta(days=1)
                                        except Exception as e:
                                            continue
                                    date = date + timedelta(days=1)
                            except Exception as e:
                                date = date + timedelta(days=1)
strategy.disconnect()

  0%|                                                                                                                                             | 0/5 [00:00<?, ?it/s]

  0%|                                                                                                                                             | 0/4 [00:00<?, ?it/s]

 25%|███████████████████████████████▊                                                                                               | 1/4 [1:09:13<3:27:41, 4153.69s/it]

 50%|███████████████████████████████████████████████████████████████▌                                                               | 2/4 [2:09:51<2:08:20, 3850.26s/it]

 75%|███████████████████████████████████████████████████████████████████████████████████████████████▎                               | 3/4 [3:19:23<1:06:36, 3997.00s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [4:34:37<0

project_strategy trades localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it, Timeout: 30s, Topology Description: <TopologyDescription id: 615fb19684a777c5dce9e8d2, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it')>]>


 25%|██████████████████████████████▌                                                                                           | 1/4 [36:03:23<108:10:09, 129803.29s/it]


KeyboardInterrupt: 